In [ ]:
import os

import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from az_ai_search.hybrid_search import cog_search

load_dotenv()  

az_oai_client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

def generate_embeddings(text, model="akm-embeddings"): # model = "deployment_name"
    return az_oai_client.embeddings.create(input = [text], model=model).data[0].embedding

def retrive_context_using_hybrid_search(question):
  vector_embeddings =  generate_embeddings(question)
  context_data = cog_search(question, vector_embeddings, [], False, [], 3, "scij-index")
  return context_data

In [1]:
from az_ai_search.hybrid_search import retrive_context_using_hybrid_search

In [2]:
query_text = "Who are the judges for TAPAS case?"
#test
context_data = retrive_context_using_hybrid_search(query_text)

{'issue_for_consideration': 'Issue for Consideration Whether gap between the retirement of a District Judge and her subsequent appointment as the Judge of the High Court would constitute ‘break in service’ adversely affecting her benefits. Headnotes The High Court Judges (Salaries and Conditions of Service) Act, 1954 – s. 14, 15, Para 2, Part III of the First Schedule – Entitlement of High Court Judge promoted from District Judiciary to pension and retirement benefits despite break in service –', 'vector': [0.014152286, -0.0003631947, 0.022536848, -0.016755773, -0.0020076886, 0.016462045, -0.010380568, -0.013150944, -0.031722482, -0.030360658, 0.0026351956, -0.0021979434, -0.015460704, -0.015701026, -0.032203127, 0.011008075, 0.03711637, 0.023738457, 0.02400548, 0.0035514226, -0.015020114, -0.0057910886, -0.0047597075, 0.007910593, -0.00058912224, 0.014405958, 0.016675664, -0.013778452, -0.00393527, 0.02291068, -0.010907941, 0.012343196, -0.034419425, -0.023831915, -0.021188375, 0.0079

In [3]:
context_data

[{'case_title': ' Union of India, Ministry of Law & Justice  Vs Justice (Retd) Raj Rahul Garg (Raj Rani Jain) and Others',
  'case_number': ' CIVIL APPEAL/4272/2024',
  'disposal_nature': ' Disposed off',
  'direction_issue': '',
  'judges': 'D.Y. CHANDRACHUD,J.B. PARDIWALA,Manoj Misra',
  'issue_for_consideration': 'Issue for Consideration Whether gap between the retirement of a District Judge and her subsequent appointment as the Judge of the High Court would constitute ‘break in service’ adversely affecting her benefits. Headnotes The High Court Judges (Salaries and Conditions of Service) Act, 1954 – s. 14, 15, Para 2, Part III of the First Schedule – Entitlement of High Court Judge promoted from District Judiciary to pension and retirement benefits despite break in service –',
  'head_notes': 'The High Court Judges (Salaries and Conditions of Service) Act, 1954 &ndash; s. 14, 15, Para 2, Part III of the First Schedule &ndash; Entitlement of High Court Judge promoted from District J

In [8]:
def generate_human_prompt(context_data):

    content = [
        f"""<CHUNK {i+1} START>
    ## Chunk Text: {item['chunk_text']}
    ## Chunk Score: {item['score']}
    <CASE METADATA START>
    ## Case Title: {item['case_title']}
    ## Judges: {item['judges']}
    ## Issue For Consideration Backgroud:  {item['issue_for_consideration']}
    <CASE METADATA END>
    <CHUNK {i+1} END>
    """
        for i,item in enumerate(context_data)
    ]

    human_prompt = f"""You can only answer questions where the answers can be found in this context given below: \n<CONTEXT START>\n"""
    human_prompt += "\n".join(content)
    human_prompt += "\n<CONTEXT END>"
    return human_prompt

human_prompt = generate_human_prompt(context_data)


In [9]:
print(human_prompt)

You can only answer questions where the answers can be found in this context given below: 
<CONTEXT START>
<CHUNK 1 START>
    ## Chunk Text: are appointed from the 
Bar under Article 217(2)(b) of the Constitution. A three-Judge Bench of 
this Court, speaking through Sathasivam, CJ noted that Judges who 
are appointed under Article 217(2)(a) being members of the judicial 
service obtain full pensionary benefits even if they serve as a Judge 
of the High Court for a bare period of a year or two because of their 
earlier entry into judicial service, but such a benefit is not extended 
to members of the Bar who become Judges of the High Court. This 
Court while laying down the principle of non-discrimination between 
High Court judges elevated from the bar on the one hand and from 
the district judiciary on the other, observed:
“19. When persons who occupied the constitutional 
office of Judge, High Court retire, there should not 
be any discrimination with regard to the fixation of 
thei

In [116]:
def generate_human_prompt_rag(question):
    context_data = retrive_context_using_hybrid_search(question)
    human_prompt = generate_human_prompt(context_data)
    return human_prompt


In [3]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_VERSION"] = os.getenv("OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
llm = AzureChatOpenAI(
    azure_deployment="akm-chat-4",  # or your deployment
    api_version="2023-06-01-preview",  # or your api version
    temperature=0
)



In [5]:
llm.invoke("what is machine learning").content

'Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform a task without using explicit instructions, relying instead on patterns and inference. It is a method of data analysis that automates analytical model building.\n\nIn machine learning, computers use data to learn for themselves. They are trained on a dataset, which they use to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are designed to improve their performance at a task over time with more data.\n\nThere are several types of machine learning, including:\n\n1. Supervised learning: The algorithm is trained on labeled data, meaning that the input data is paired with the correct output. The algorithm learns to map inputs to outputs and can then predict the output for new, unseen data.\n\n2. Unsupervised learning: The algorithm is given data without any label

In [118]:
class LegalResponseParser(BaseOutputParser[dict]):
    """Custom parser for legal case responses using regex."""

    def parse(self, text: str) -> dict:
        # Define regex patterns for extracting the fields
        answer_pattern = re.compile(r"'answer':\s*\"([^\"]*)\"", re.DOTALL)
        follow_up_pattern = re.compile(r"'follow_up_questions':\s*\[(.*?)\]", re.DOTALL)
        
        # Extract the answer
        answer_match = answer_pattern.search(text)
        if not answer_match:
            raise OutputParserException("Could not find 'answer' in the response.")
        answer = answer_match.group(1).strip()

        # Extract the follow-up questions if present
        follow_up_questions = []
        follow_up_match = follow_up_pattern.search(text)
        if follow_up_match:
            follow_up_text = follow_up_match.group(1).strip()
            follow_up_questions = [q.strip().strip('"') for q in follow_up_text.split(',')]
        
        # Construct the final dictionary
        response_dict = {
            "answer": answer,
            "follow_up_questions": follow_up_questions
        }

        return response_dict

    @property
    def _type(self) -> str:
        return "legal_response_parser"

In [119]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
# Define the system and human message templates
system_message = SystemMessagePromptTemplate.from_template(
    """
Assistant helps the user with their questions related to legal research. You are knowledgeable in the legal domain and aim to answer the user's questions accurately, relying on case law, statutes, and legal principles. Only provide a response following the specified format without deviation. If you do not know the answer, return ['answer': "None"]. Do not provide incorrect answers. If asking a clarifying question to the user would help, ask the question.

If the user asks a question and you do not know the answer, return ['answer': "None"].

# Desired Output

Use this format to frame your answers:
Only provide a response following this format without deviation.

[
    'answer': "the answer to the user's question, specific to the legal domain",
    'follow_up_questions': [ 
        "a comma separated list of three follow up questions that can be asked to build on your answer, related to legal research"
    ]
]
You must only use this format; you are not allowed to change the format.

# List of Examples

EXAMPLE
Conversation history:
USER: What is the doctrine of stare decisis?
ASSISTANT: [
    'answer': "The doctrine of stare decisis is a legal principle that courts should follow precedents established in previous cases when making rulings in similar cases. It ensures consistency and stability in the law.",
    'follow_up_questions': [
        "What are some landmark cases that established important precedents under the doctrine of stare decisis?",
        "How does the doctrine of stare decisis influence judicial decision-making in different jurisdictions?",
        "Can the doctrine of stare decisis ever be overturned or disregarded by a court?"
    ]
]
END OF EXAMPLE

EXAMPLE
Conversation history:
USER: What are the key differences between civil and criminal law?
ASSISTANT: [
    'answer': "Civil law deals with disputes between individuals or organizations, typically involving private rights, whereas criminal law involves offenses against the state or public, where the government prosecutes the offender. The burden of proof is higher in criminal cases, requiring proof beyond a reasonable doubt, while civil cases require a preponderance of evidence.",
    'follow_up_questions': [ 
        "What are some examples of cases that would fall under civil law?",
        "How does the burden of proof differ between civil and criminal cases?",
        "What are the potential penalties or remedies in civil law compared to criminal law?"
    ]
]
END OF EXAMPLE

EXAMPLE
Conversation history:
USER: Can a contract be enforced if it was made under duress?
ASSISTANT: [
    'answer': "A contract made under duress is generally considered voidable, meaning the party who was forced into the agreement can choose to either enforce or void the contract. Duress involves coercion or threats that deprive a party of free will in entering into the contract.",
    'follow_up_questions': [
        "What legal remedies are available for a party seeking to void a contract made under duress?",
        "How do courts determine whether duress was present when a contract was formed?",
        "Are there any exceptions where a contract made under duress might still be enforceable?"
    ]
]
END OF EXAMPLE

EXAMPLE
Conversation history:
USER: What's the current population of the United States?
ASSISTANT: ['answer': "None"]
END OF EXAMPLE

# Return ['answer': "None"] if the question cannot be answered.
# Return in the desired list format if you can answer.
"""
)

In [120]:

def answer_question(input_question):

    human_prompt = generate_human_prompt_rag(input_question)
    human_message = HumanMessagePromptTemplate.from_template(human_prompt+"{question}")

    # Create the ChatPromptTemplate from the messages
    prompt = ChatPromptTemplate.from_messages([system_message, human_message])
    output_parser = LegalResponseParser()
    chain = prompt|llm|output_parser
    # Now you can use this prompt template in your chain or model invocation
    llm_response = chain.invoke({"question": input_question})
    return llm_response
answer_question("What is the issue for SBI?")

{'content_group': None, 'index_key': 'b857626ba8a5_aHR0cHM6Ly9zY2lnZW5haWRhdGEuYmxvYi5jb3JlLndpbmRvd3MubmV0L3NjaS1kYXRhL3NjaS1kb2N1bWVudHMvTUlTQ0VMTEFORU9VUyUyMEFQUExJQ0FUSU9OXzQ4Nl8yMDI0LnBkZg2_chunks_0', 'slide_count': None, 'content_encoding': None, 'authors': [], 'head_notes': None, 'disposal_nature': None, 'metadata_storage_name': 'MISCELLANEOUS APPLICATION_486_2024.pdf', 'page_count': None, 'issue_for_consideration': None, 'document_id': None, 'metadata_storage_content_md5': None, 'last_modified': None, 'document_url': None, 'case_number': None, 'direction_issue': None, 'description': None, 'title': None, 'chunk': '[2024] 3 S.C.R. 637 : 2024 INSC 195\n\nState Bank of India \nv.  \n\nAssociation for Democratic Reforms and Others\n(Miscellaneous Application No 486 of 2024 \n\nIn \nWrit Petition (Civil) No 880 of 2017) \n\nWITH \n(Contempt Petition (Civil) No 138 of 2024 \n\nIn \nWrit Petition (Civil) No 880 of 2017) \n\n& \n(Contempt Petition (Civil) No 140 of 2024 \n\nIn \nWrit Pe

{'answer': 'The issue for the State Bank of India (SBI) pertains to a Miscellaneous Application filed by them seeking an extension of time until 30 June 2024 for complying with the directions given by the Supreme Court in its judgment dated 15 February 2024. The Court had directed the SBI to submit to the Election Commission of India (ECI) details of the Electoral Bonds purchased by contributors and redeemed by political parties between 12 April 2019 till the date of the judgment, by 6 March 2024. The SBI is facing difficulty in disclosing information which is readily available with it and is seeking an extension of time.',
 'follow_up_questions': ['What are Electoral Bonds and how do they work?',
  "What was the Supreme Court's judgment dated 15 February 2024 about?",
  "What are the potential consequences for SBI if they fail to comply with the court's directions?"]}

In [121]:
language_codes = {
    "Assamese": "as",
    "Bengali": "bn",
    "Bodo": "brx",
    "Dogri": "doi",
    "Gujarati": "gu",
    "Hindi": "hi",
    "Kannada": "kn",
    "Kashmiri": "ks",
    "Konkani": "kok",
    "Maithili": "mai",
    "Malayalam": "ml",
    "Manipuri": "mni",
    "Marathi": "mr",
    "Nepali": "ne",
    "Odia": "or",
    "Punjabi": "pa",
    "Sanskrit": "sa",
    "Santali": "sat",
    "Sindhi": "sd",
    "Tamil": "ta",
    "Telugu": "te",
    "Urdu": "ur"
}


In [ ]:
# This example requires environment variables named "LANGUAGE_KEY" and "LANGUAGE_ENDPOINT"

from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

cog_service_endpoint = "https://vsent-sub-cog-services.cognitiveservices.azure.com/"
cog_service_key = ""

credential = AzureKeyCredential(cog_service_key)
language_detection_client = TextAnalyticsClient(endpoint=cog_service_endpoint, credential=credential)
# Example method for detecting the language of text
def language_detection_example(documents):
    try:
        response = language_detection_client.detect_language(documents = documents, country_hint = 'us')[0]
        print("Language: ", response.primary_language.name)
        return response.primary_language.iso6391_name

    except Exception as err:
        print("Encountered exception. {}".format(err))
resp = language_detection_example(["machine learning is the best"])

Language:  English


'en'

In [35]:
from azure.ai.translation.text import TextTranslationClient
# Initialize the client
credential = AzureKeyCredential(cog_service_key)
text_translator = TextTranslationClient(endpoint=cog_service_endpoint, credential=credential)

def translate_text(input_text_elements, to_language):
    try:

        response = text_translator.translate(body=input_text_elements, to_language=to_language)
        translations = response[0:] if response else None

        output_text_elements = []

        for translation in translations:
            if translation:
                detected_language = translation.detected_language
                if detected_language:
                    print(
                        f"Detected languages of the input text: {detected_language.language} with score: {detected_language.score}."
                    )
                for translated_text in translation.translations:
                    output_text_elements.append(translated_text.text)
                    print(f"Text was translated to: '{translated_text.to}' and the result is: '{translated_text.text}'.")

        return output_text_elements

    except Exception as exception:
        print(exception)

In [133]:
TextTranslationClient()

In [17]:
data = {'answer': 'The issue for consideration in the SEBI case pertains to the Adani-Hindenburg report alleging that the Adani Group manipulated its share price, with the petitioner seeking investigation by the Special Investigation Team or by the CBI. The case involves allegations of regulatory non-compliance, legal violations, and failure to disclose transactions with related parties and other relevant information in violation of the regulations.', 'follow_up_questions': ['What are the specific regulations and laws alleged to have been violated by the Adani Group?', 'How has SEBI responded to the allegations and what actions has it taken in this case?', "What are the potential implications of the findings in this case on the Adani Group and SEBI's regulatory practices?"]}

In [20]:
temp = [data['answer']]
temp.extend(data['follow_up_questions'])
temp

['The issue for consideration in the SEBI case pertains to the Adani-Hindenburg report alleging that the Adani Group manipulated its share price, with the petitioner seeking investigation by the Special Investigation Team or by the CBI. The case involves allegations of regulatory non-compliance, legal violations, and failure to disclose transactions with related parties and other relevant information in violation of the regulations.',
 'What are the specific regulations and laws alleged to have been violated by the Adani Group?',
 'How has SEBI responded to the allegations and what actions has it taken in this case?',
 "What are the potential implications of the findings in this case on the Adani Group and SEBI's regulatory practices?"]

In [36]:
translate_text(temp, ["kn"])

Detected languages of the input text: en with score: 1.0.
Text was translated to: 'kn' and the result is: 'ಸೆಬಿ ಪ್ರಕರಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕಾದ ವಿಷಯವು ಅದಾನಿ ಗ್ರೂಪ್ ತನ್ನ ಷೇರು ಬೆಲೆಯನ್ನು ತಿರುಚಿದೆ ಎಂದು ಆರೋಪಿಸಿದ ಅದಾನಿ-ಹಿಂಡೆನ್ಬರ್ಗ್ ವರದಿಗೆ ಸಂಬಂಧಿಸಿದೆ, ಅರ್ಜಿದಾರರು ವಿಶೇಷ ತನಿಖಾ ತಂಡ ಅಥವಾ ಸಿಬಿಐನಿಂದ ತನಿಖೆಯನ್ನು ಕೋರಿದ್ದಾರೆ. ಈ ಪ್ರಕರಣವು ನಿಯಂತ್ರಕ ಅನುಸರಣೆ, ಕಾನೂನು ಉಲ್ಲಂಘನೆಗಳು ಮತ್ತು ನಿಯಮಗಳನ್ನು ಉಲ್ಲಂಘಿಸಿ ಸಂಬಂಧಿತ ಪಕ್ಷಗಳೊಂದಿಗೆ ವಹಿವಾಟುಗಳು ಮತ್ತು ಇತರ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಬಹಿರಂಗಪಡಿಸಲು ವಿಫಲವಾದ ಆರೋಪಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.'.
Detected languages of the input text: en with score: 1.0.
Text was translated to: 'kn' and the result is: 'ಅದಾನಿ ಗ್ರೂಪ್ ಉಲ್ಲಂಘಿಸಿದೆ ಎಂದು ಹೇಳಲಾದ ನಿರ್ದಿಷ್ಟ ನಿಯಮಗಳು ಮತ್ತು ಕಾನೂನುಗಳು ಯಾವುವು?'.
Detected languages of the input text: en with score: 1.0.
Text was translated to: 'kn' and the result is: 'ಆರೋಪಗಳಿಗೆ ಸೆಬಿ ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸಿದೆ ಮತ್ತು ಈ ಪ್ರಕರಣದಲ್ಲಿ ಅದು ಯಾವ ಕ್ರಮಗಳನ್ನು ತೆಗೆದುಕೊಂಡಿದೆ?'.
Detected languages of the input text: en with score: 1.0.
Text was translated to: 'kn' and the result is: 'ಅದಾನಿ ಗ್ರೂಪ್ ಮತ್ತು ಸೆ

['ಸೆಬಿ ಪ್ರಕರಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕಾದ ವಿಷಯವು ಅದಾನಿ ಗ್ರೂಪ್ ತನ್ನ ಷೇರು ಬೆಲೆಯನ್ನು ತಿರುಚಿದೆ ಎಂದು ಆರೋಪಿಸಿದ ಅದಾನಿ-ಹಿಂಡೆನ್ಬರ್ಗ್ ವರದಿಗೆ ಸಂಬಂಧಿಸಿದೆ, ಅರ್ಜಿದಾರರು ವಿಶೇಷ ತನಿಖಾ ತಂಡ ಅಥವಾ ಸಿಬಿಐನಿಂದ ತನಿಖೆಯನ್ನು ಕೋರಿದ್ದಾರೆ. ಈ ಪ್ರಕರಣವು ನಿಯಂತ್ರಕ ಅನುಸರಣೆ, ಕಾನೂನು ಉಲ್ಲಂಘನೆಗಳು ಮತ್ತು ನಿಯಮಗಳನ್ನು ಉಲ್ಲಂಘಿಸಿ ಸಂಬಂಧಿತ ಪಕ್ಷಗಳೊಂದಿಗೆ ವಹಿವಾಟುಗಳು ಮತ್ತು ಇತರ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಬಹಿರಂಗಪಡಿಸಲು ವಿಫಲವಾದ ಆರೋಪಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.',
 'ಅದಾನಿ ಗ್ರೂಪ್ ಉಲ್ಲಂಘಿಸಿದೆ ಎಂದು ಹೇಳಲಾದ ನಿರ್ದಿಷ್ಟ ನಿಯಮಗಳು ಮತ್ತು ಕಾನೂನುಗಳು ಯಾವುವು?',
 'ಆರೋಪಗಳಿಗೆ ಸೆಬಿ ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸಿದೆ ಮತ್ತು ಈ ಪ್ರಕರಣದಲ್ಲಿ ಅದು ಯಾವ ಕ್ರಮಗಳನ್ನು ತೆಗೆದುಕೊಂಡಿದೆ?',
 'ಅದಾನಿ ಗ್ರೂಪ್ ಮತ್ತು ಸೆಬಿಯ ನಿಯಂತ್ರಕ ಅಭ್ಯಾಸಗಳ ಮೇಲೆ ಈ ಪ್ರಕರಣದ ಸಂಶೋಧನೆಗಳ ಸಂಭಾವ್ಯ ಪರಿಣಾಮಗಳು ಯಾವುವು?']

In [34]:
res = ['ಸೆಬಿ ಪ್ರಕರಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕಾದ ವಿಷಯವು ಅದಾನಿ ಗ್ರೂಪ್ ತನ್ನ ಷೇರು ಬೆಲೆಯನ್ನು ತಿರುಚಿದೆ ಎಂದು ಆರೋಪಿಸಿದ ಅದಾನಿ-ಹಿಂಡೆನ್ಬರ್ಗ್ ವರದಿಗೆ ಸಂಬಂಧಿಸಿದೆ, ಅರ್ಜಿದಾರರು ವಿಶೇಷ ತನಿಖಾ ತಂಡ ಅಥವಾ ಸಿಬಿಐನಿಂದ ತನಿಖೆಯನ್ನು ಕೋರಿದ್ದಾರೆ. ಈ ಪ್ರಕರಣವು ನಿಯಂತ್ರಕ ಅನುಸರಣೆ, ಕಾನೂನು ಉಲ್ಲಂಘನೆಗಳು ಮತ್ತು ನಿಯಮಗಳನ್ನು ಉಲ್ಲಂಘಿಸಿ ಸಂಬಂಧಿತ ಪಕ್ಷಗಳೊಂದಿಗೆ ವಹಿವಾಟುಗಳು ಮತ್ತು ಇತರ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಬಹಿರಂಗಪಡಿಸಲು ವಿಫಲವಾದ ಆರೋಪಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.']
for re in res:
    print(re)

ಸೆಬಿ ಪ್ರಕರಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕಾದ ವಿಷಯವು ಅದಾನಿ ಗ್ರೂಪ್ ತನ್ನ ಷೇರು ಬೆಲೆಯನ್ನು ತಿರುಚಿದೆ ಎಂದು ಆರೋಪಿಸಿದ ಅದಾನಿ-ಹಿಂಡೆನ್ಬರ್ಗ್ ವರದಿಗೆ ಸಂಬಂಧಿಸಿದೆ, ಅರ್ಜಿದಾರರು ವಿಶೇಷ ತನಿಖಾ ತಂಡ ಅಥವಾ ಸಿಬಿಐನಿಂದ ತನಿಖೆಯನ್ನು ಕೋರಿದ್ದಾರೆ. ಈ ಪ್ರಕರಣವು ನಿಯಂತ್ರಕ ಅನುಸರಣೆ, ಕಾನೂನು ಉಲ್ಲಂಘನೆಗಳು ಮತ್ತು ನಿಯಮಗಳನ್ನು ಉಲ್ಲಂಘಿಸಿ ಸಂಬಂಧಿತ ಪಕ್ಷಗಳೊಂದಿಗೆ ವಹಿವಾಟುಗಳು ಮತ್ತು ಇತರ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಬಹಿರಂಗಪಡಿಸಲು ವಿಫಲವಾದ ಆರೋಪಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.


In [61]:
response = ['ಸೆಬಿ ಪ್ರಕರಣದಲ್ಲಿ ಪರಿಗಣಿಸಬೇಕಾದ ವಿಷಯವು ಅದಾನಿ ಗ್ರೂಪ್ ತನ್ನ ಷೇರು ಬೆಲೆಯನ್ನು ತಿರುಚಿದೆ ಎಂದು ಆರೋಪಿಸಿದ ಅದಾನಿ-ಹಿಂಡೆನ್ಬರ್ಗ್ ವರದಿಗೆ ಸಂಬಂಧಿಸಿದೆ, ಅರ್ಜಿದಾರರು ವಿಶೇಷ ತನಿಖಾ ತಂಡ ಅಥವಾ ಸಿಬಿಐನಿಂದ ತನಿಖೆಯನ್ನು ಕೋರಿದ್ದಾರೆ. ಈ ಪ್ರಕರಣವು ನಿಯಂತ್ರಕ ಅನುಸರಣೆ, ಕಾನೂನು ಉಲ್ಲಂಘನೆಗಳು ಮತ್ತು ನಿಯಮಗಳನ್ನು ಉಲ್ಲಂಘಿಸಿ ಸಂಬಂಧಿತ ಪಕ್ಷಗಳೊಂದಿಗೆ ವಹಿವಾಟುಗಳು ಮತ್ತು ಇತರ ಸಂಬಂಧಿತ ಮಾಹಿತಿಯನ್ನು ಬಹಿರಂಗಪಡಿಸಲು ವಿಫಲವಾದ ಆರೋಪಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.',
 'ಅದಾನಿ ಗ್ರೂಪ್ ಉಲ್ಲಂಘಿಸಿದೆ ಎಂದು ಹೇಳಲಾದ ನಿರ್ದಿಷ್ಟ ನಿಯಮಗಳು ಮತ್ತು ಕಾನೂನುಗಳು ಯಾವುವು?',
 'ಆರೋಪಗಳಿಗೆ ಸೆಬಿ ಹೇಗೆ ಪ್ರತಿಕ್ರಿಯಿಸಿದೆ ಮತ್ತು ಈ ಪ್ರಕರಣದಲ್ಲಿ ಅದು ಯಾವ ಕ್ರಮಗಳನ್ನು ತೆಗೆದುಕೊಂಡಿದೆ?',
 'ಅದಾನಿ ಗ್ರೂಪ್ ಮತ್ತು ಸೆಬಿಯ ನಿಯಂತ್ರಕ ಅಭ್ಯಾಸಗಳ ಮೇಲೆ ಈ ಪ್ರಕರಣದ ಸಂಶೋಧನೆಗಳ ಸಂಭಾವ್ಯ ಪರಿಣಾಮಗಳು ಯಾವುವು?']

def format_response (response, summary=False):
    if summary:
         return f"**SUMMARY**\n{response['summary']}"  
    else:
         if response['answer'] == "None":
              return "Im sorry, Im Unable to answer that question"
         res = [response['answer']]
         res.extend(response['follow_up_questions'])
         return f"{res[0]}\n\nSuggested follow up questions:\n{res[1]}\n{res[2]}\n{res[3]}"
         

data = {'answer': 'The issue for consideration in the SEBI case pertains to the Adani-Hindenburg report alleging that the Adani Group manipulated its share price, with the petitioner seeking investigation by the Special Investigation Team or by the CBI. The case involves allegations of regulatory non-compliance, legal violations, and failure to disclose transactions with related parties and other relevant information in violation of the regulations.', 'follow_up_questions': ['What are the specific regulations and laws alleged to have been violated by the Adani Group?', 'How has SEBI responded to the allegations and what actions has it taken in this case?', "What are the potential implications of the findings in this case on the Adani Group and SEBI's regulatory practices?"]}
data = {'summary' : 'summary'}
print(format_response(data, True))

**SUMMARY**
summary


In [46]:
len(response)

4

In [ ]:
#get summary for case number

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, QueryLanguage
# from azure.search.documents.models import VectorizableTextQuery

def fetch_summary(case_number):
    service_endpoint: str = "https://vs-end-sub-aisearch.search.windows.net"
    key: str = ""
    index_name: str = "scij-index"
    
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

    results_list = search_client.search(  
                "*",    
                query_type=QueryType.SIMPLE,
                query_language=QueryLanguage.EN_US, 
                #select=["judgement_sumary"],
                filter="parent_id eq null and case_number eq ' {case_number}'",   
                semantic_configuration_name="default",  
                
            )
    # Initialize a list to store the results as dictionaries
    
    for item in results_list:
        return {"summary" : item['judgement_summary']}

cog_search()


{'summary': "### Summary of the Judgment\r\n\r\n**Case Title:** Bloomberg Television Production Services India Private Limited & Ors. v. Zee Entertainment Enterprises Limited  \r\n**Citation:** [2024] 3 S.C.R. 994 : 2024 INSC 255  \r\n**Court:** Supreme Court of India  \r\n**Bench:** Dr. Dhananjaya Y Chandrachud, CJI, J B Pardiwala, and Manoj Misra, JJ.  \r\n**Date of Judgment:** 22 March 2024  \r\n**Civil Appeal No.:** 4602 of 2024\r\n\r\n#### Issue for Consideration\r\nThe primary issue in this case was the validity of an interim order passed by the trial judge, which was upheld by the High Court. The order directed the appellants, a media platform, to take down an article published against the respondent and restrained them from posting, circulating, or publishing the article on any platform until the next hearing.\r\n\r\n#### Key Information Driving the Judgment Decision\r\n1. **Lack of Reasoning in the Trial Judge's Order:**\r\n   - The trial judge's order did not discuss the prim

In [72]:
res.values()

AttributeError: 'NoneType' object has no attribute 'values'